In [5]:
"""
yui2024式 歌唱合成ソフト v9.0 — Google Colab 完全単体版
==========================================================
kana_map.py / ut_one.py をこのファイルに完全に内包しています。
アップロード不要。このファイル1つをセルに貼り付けるか
%run で実行するだけで動作します。

【実行方法】
  このファイルを Colab にアップロードして:
      %run singing_synth_colab.py
  または中身をそのままセルに貼り付けて実行

【モデルファイルの読み込み方法 (どちらか)】
  A) Google Drive をマウントしてパスを入力:
       from google.colab import drive; drive.mount('/content/drive')
       パス例: /content/drive/MyDrive/model.yvs

  B) Colab のサイドバーからモデルファイルをアップロード:
       パス例: /content/model.yvs
"""

# =============================================================================
# 1. インストール
# =============================================================================
import subprocess, sys
for _pkg in ["gradio", "pyworld", "librosa", "soundfile", "scipy"]:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", _pkg])

# =============================================================================
# 2. インポート
# =============================================================================
import os, tempfile, traceback, pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import soundfile as sf
import librosa
import pyworld as pw
from scipy import signal
from scipy.ndimage import gaussian_filter1d
from scipy.signal import medfilt
from scipy.interpolate import interp1d
import gradio as gr

try:
    from torch.nn.utils.parametrizations import weight_norm
except ImportError:
    from torch.nn.utils import weight_norm
# =============================================================================
# ★ モデル登録テーブル ★
# =============================================================================
# ここにモデルを登録してください。
# 形式: "表示名": "Google Drive 上のフルパス (.yvs / .pth)"
#
# 【追加方法】
#   1. このテーブルに新しい行を 1 行追加するだけです。
#   2. 例:
#        "歌手名A": "/content/drive/MyDrive/models/singer_a.yvs",
#
# ⚠ Google Drive を使う場合は事前に以下をセルで実行してください:
#      from google.colab import drive; drive.mount("/content/drive")
# =============================================================================

MODEL_REGISTRY = {
    # ↓ ここから登録 ↓ ------------------------------------------------
    "yui2024": "/content/drive/MyDrive/model/yui.yvs",
    "波音リツ": "/content/drive/MyDrive/model/base_model.yvs",
    # ↑ ここまで登録 ↑ ------------------------------------------------
}

# ドロップダウン用の選択肢リスト（自動生成・編集不要）
_MODEL_CHOICES = list(MODEL_REGISTRY.keys())



# =============================================================================
# 3. kana_map.py — 完全内包 (ファイルそのまま)
# =============================================================================
# Manual Kana-to-Phoneme Mapping for Singing Voice Synthesis
# Matches pyopenjtalk output format where possible.
# v9.0: Fixed mixed-language keys

KANA_MAPPING = {
    # Basic Vowels
    'あ': ['a'], 'い': ['i'], 'う': ['u'], 'え': ['e'], 'お': ['o'],
    'ア': ['a'], 'イ': ['i'], 'ウ': ['u'], 'エ': ['e'], 'オ': ['o'],

    # K-line
    'か': ['k', 'a'], 'き': ['k', 'i'], 'く': ['k', 'u'], 'け': ['k', 'e'], 'こ': ['k', 'o'],
    'カ': ['k', 'a'], 'キ': ['k', 'i'], 'ク': ['k', 'u'], 'ケ': ['k', 'e'], 'コ': ['k', 'o'],

    # S-line
    'さ': ['s', 'a'], 'し': ['sh', 'i'], 'す': ['s', 'u'], 'せ': ['s', 'e'], 'そ': ['s', 'o'],
    'サ': ['s', 'a'], 'シ': ['sh', 'i'], 'ス': ['s', 'u'], 'セ': ['s', 'e'], 'ソ': ['s', 'o'],

    # T-line
    'た': ['t', 'a'], 'ち': ['ch', 'i'], 'つ': ['ts', 'u'], 'て': ['t', 'e'], 'と': ['t', 'o'],
    'タ': ['t', 'a'], 'チ': ['ch', 'i'], 'ツ': ['ts', 'u'], 'テ': ['t', 'e'], 'ト': ['t', 'o'],

    # N-line
    'な': ['n', 'a'], 'に': ['n', 'i'], 'ぬ': ['n', 'u'], 'ね': ['n', 'e'], 'の': ['n', 'o'],
    'ナ': ['n', 'a'], 'ニ': ['n', 'i'], 'ヌ': ['n', 'u'], 'ネ': ['n', 'e'], 'ノ': ['n', 'o'],

    # H-line
    'は': ['h', 'a'], 'ひ': ['h', 'i'], 'ふ': ['f', 'u'], 'へ': ['h', 'e'], 'ほ': ['h', 'o'],
    'ハ': ['h', 'a'], 'ヒ': ['h', 'i'], 'フ': ['f', 'u'], 'ヘ': ['h', 'e'], 'ホ': ['h', 'o'],

    # M-line
    'ま': ['m', 'a'], 'み': ['m', 'i'], 'む': ['m', 'u'], 'め': ['m', 'e'], 'も': ['m', 'o'],
    'マ': ['m', 'a'], 'ミ': ['m', 'i'], 'ム': ['m', 'u'], 'メ': ['m', 'e'], 'モ': ['m', 'o'],

    # Y-line
    'や': ['y', 'a'], 'ゆ': ['y', 'u'], 'よ': ['y', 'o'],
    'ヤ': ['y', 'a'], 'ユ': ['y', 'u'], 'ヨ': ['y', 'o'],

    # R-line
    'ら': ['r', 'a'], 'り': ['r', 'i'], 'る': ['r', 'u'], 'れ': ['r', 'e'], 'ろ': ['r', 'o'],
    'ラ': ['r', 'a'], 'リ': ['r', 'i'], 'ル': ['r', 'u'], 'レ': ['r', 'e'], 'ロ': ['r', 'o'],

    # W-line
    'わ': ['w', 'a'], 'を': ['o'], 'ん': ['N'],
    'ワ': ['w', 'a'], 'ヲ': ['o'], 'ン': ['N'],

    # G-line (Dakuten)
    'が': ['g', 'a'], 'ぎ': ['g', 'i'], 'ぐ': ['g', 'u'], 'げ': ['g', 'e'], 'ご': ['g', 'o'],
    'ガ': ['g', 'a'], 'ギ': ['g', 'i'], 'グ': ['g', 'u'], 'ゲ': ['g', 'e'], 'ゴ': ['g', 'o'],

    # Z-line (Dakuten)
    'ざ': ['z', 'a'], 'じ': ['j', 'i'], 'ず': ['z', 'u'], 'ぜ': ['z', 'e'], 'ぞ': ['z', 'o'],
    'ザ': ['z', 'a'], 'ジ': ['j', 'i'], 'ズ': ['z', 'u'], 'ゼ': ['z', 'e'], 'ゾ': ['z', 'o'],
    'じぇ': ['j', 'e'], 'ジェ': ['j', 'e'],

    # D-line (Dakuten)
    'だ': ['d', 'a'], 'ぢ': ['j', 'i'], 'づ': ['z', 'u'], 'で': ['d', 'e'], 'ど': ['d', 'o'],
    'ダ': ['d', 'a'], 'ヂ': ['j', 'i'], 'ヅ': ['z', 'u'], 'デ': ['d', 'e'], 'ド': ['d', 'o'],

    # B-line (Dakuten)
    'ば': ['b', 'a'], 'び': ['b', 'i'], 'ぶ': ['b', 'u'], 'べ': ['b', 'e'], 'ぼ': ['b', 'o'],
    'バ': ['b', 'a'], 'ビ': ['b', 'i'], 'ブ': ['b', 'u'], 'ベ': ['b', 'e'], 'ボ': ['b', 'o'],

    # P-line (Handakuten)
    'ぱ': ['p', 'a'], 'ぴ': ['p', 'i'], 'ぷ': ['p', 'u'], 'ぺ': ['p', 'e'], 'ぽ': ['p', 'o'],
    'パ': ['p', 'a'], 'ピ': ['p', 'i'], 'プ': ['p', 'u'], 'ペ': ['p', 'e'], 'ポ': ['p', 'o'],

    # Yoon (Contracted sounds) - Fixed: all keys are now kana
    'きゃ': ['ky', 'a'], 'きゅ': ['ky', 'u'], 'きぇ': ['ky', 'e'], 'きょ': ['ky', 'o'],
    'キャ': ['ky', 'a'], 'キュ': ['ky', 'u'], 'キェ': ['ky', 'e'], 'キョ': ['ky', 'o'],

    'しゃ': ['sh', 'a'], 'しゅ': ['sh', 'u'], 'しぇ': ['sh', 'e'], 'しょ': ['sh', 'o'],
    'シャ': ['sh', 'a'], 'シュ': ['sh', 'u'], 'シェ': ['sh', 'e'], 'ショ': ['sh', 'o'],

    'ちゃ': ['ch', 'a'], 'ちゅ': ['ch', 'u'], 'ちぇ': ['ch', 'e'], 'ちょ': ['ch', 'o'],
    'チャ': ['ch', 'a'], 'チュ': ['ch', 'u'], 'チェ': ['ch', 'e'], 'チョ': ['ch', 'o'],

    'にゃ': ['ny', 'a'], 'にゅ': ['ny', 'u'], 'にぇ': ['ny', 'e'], 'にょ': ['ny', 'o'],
    'ニャ': ['ny', 'a'], 'ニュ': ['ny', 'u'], 'ニェ': ['ny', 'e'], 'ニョ': ['ny', 'o'],

    'ひゃ': ['hy', 'a'], 'ひゅ': ['hy', 'u'], 'ひぇ': ['hy', 'e'], 'ひょ': ['hy', 'o'],
    'ヒャ': ['hy', 'a'], 'ヒュ': ['hy', 'u'], 'ヒェ': ['hy', 'e'], 'ヒョ': ['hy', 'o'],

    'みゃ': ['my', 'a'], 'みゅ': ['my', 'u'], 'みぇ': ['my', 'e'], 'みょ': ['my', 'o'],
    'ミャ': ['my', 'a'], 'ミュ': ['my', 'u'], 'ミェ': ['my', 'e'], 'ミョ': ['my', 'o'],

    'りゃ': ['ry', 'a'], 'りゅ': ['ry', 'u'], 'りぇ': ['ry', 'e'], 'りょ': ['ry', 'o'],
    'リャ': ['ry', 'a'], 'リュ': ['ry', 'u'], 'リェ': ['ry', 'e'], 'リョ': ['ry', 'o'],

    'ぎゃ': ['gy', 'a'], 'ぎゅ': ['gy', 'u'], 'ぎぇ': ['gy', 'e'], 'ぎょ': ['gy', 'o'],
    'ギャ': ['gy', 'a'], 'ギュ': ['gy', 'u'], 'ギェ': ['gy', 'e'], 'ギョ': ['gy', 'o'],

    'じゃ': ['j', 'a'], 'じゅ': ['j', 'u'], 'じぇ': ['j', 'e'], 'じょ': ['j', 'o'],
    'ジャ': ['j', 'a'], 'ジュ': ['j', 'u'], 'ジェ': ['j', 'e'], 'ジョ': ['j', 'o'],

    'ぢゃ': ['j', 'a'], 'ぢゅ': ['j', 'u'], 'ぢぇ': ['j', 'e'], 'ぢょ': ['j', 'o'],
    'ヂャ': ['j', 'a'], 'ヂュ': ['j', 'u'], 'ヂェ': ['j', 'e'], 'ヂョ': ['j', 'o'],

    'でゃ': ['dy', 'a'], 'でゅ': ['dy', 'u'], 'でぇ': ['dy', 'e'], 'でょ': ['dy', 'o'],
    'デャ': ['dy', 'a'], 'デュ': ['dy', 'u'], 'デェ': ['dy', 'e'], 'デョ': ['dy', 'o'],

    'びゃ': ['by', 'a'], 'びゅ': ['by', 'u'], 'びぇ': ['by', 'e'], 'びょ': ['by', 'o'],
    'ビャ': ['by', 'a'], 'ビュ': ['by', 'u'], 'ビェ': ['by', 'e'], 'ビョ': ['by', 'o'],

    'ぴゃ': ['py', 'a'], 'ぴゅ': ['py', 'u'], 'ぴぇ': ['py', 'e'], 'ぴょ': ['py', 'o'],
    'ピャ': ['py', 'a'], 'ピュ': ['py', 'u'], 'ピェ': ['py', 'e'], 'ピョ': ['py', 'o'],

    # Extra / Special
    'ふぁ': ['f', 'a'], 'ふぃ': ['f', 'i'], 'ふぇ': ['f', 'e'], 'ふぉ': ['f', 'o'],
    'ファ': ['f', 'a'], 'フィ': ['f', 'i'], 'フェ': ['f', 'e'], 'フォ': ['f', 'o'],

    'うぃ': ['w', 'i'], 'うぇ': ['w', 'e'], 'うぉ': ['w', 'o'],
    'ウィ': ['w', 'i'], 'ウェ': ['w', 'e'], 'ウォ': ['w', 'o'],

    'ヴ': ['b', 'u'], 'ゔ': ['b', 'u'],
    'ヴァ': ['b', 'a'], 'ヴィ': ['b', 'i'], 'ヴェ': ['b', 'e'], 'ヴォ': ['b', 'o'],

    'ゐ': ['w', 'i'], 'ヰ': ['w', 'i'],
    'ゑ': ['w', 'e'], 'ヱ': ['w', 'e'],
    'ヶ': ['k', 'e'], 'ヵ': ['k', 'a'],
    'ゎ': ['w', 'a'], 'ヮ': ['w', 'a'],

    'てぃ': ['t', 'i'], 'てゅ': ['t', 'u'], 'でぃ': ['d', 'i'], 'でゅ': ['d', 'u'],
    'ティ': ['t', 'i'], 'テュ': ['t', 'u'], 'ディ': ['d', 'i'], 'デュ': ['d', 'u'],

    'とぅ': ['t', 'u'], 'どぅ': ['d', 'u'],
    'トゥ': ['t', 'u'], 'ドゥ': ['d', 'u'],

    # Sokuon / Pause
    'っ': ['cl'], 'ッ': ['cl'],
    ' ': ['pau'], '　': ['pau'], '、': ['pau'], '。': ['pau'],
}

# Sorted keys by length (descending) to match compound sounds first
SORTED_KEYS = sorted(KANA_MAPPING.keys(), key=len, reverse=True)

def g2p_manual(text):
    if not text: return ['SP']

    # Normalize
    text = text.replace('pau', ' ').replace('SP', ' ')

    phonemes = []
    i = 0
    while i < len(text):
        matched = False
        for k in SORTED_KEYS:
            if text[i:].startswith(k):
                phonemes.extend(KANA_MAPPING[k])
                i += len(k)
                matched = True
                break

        if not matched:
            char = text[i]
            if 'a' <= char <= 'z' or 'A' <= char <= 'Z':
                pass
            elif char in [' ', '　']:
                phonemes.append('SP')
            i += 1

    if not phonemes: return ['SP']
    return phonemes


# =============================================================================
# 4. ut_one.py — 完全内包 (ファイルそのまま、kana_map 依存を上記定義で解決済み)
# =============================================================================
"""
ut_one.py ─ 歌声合成コアライブラリ（v9.0 プロ品質版）
SynthesizerV / NEUTRINO / sinsy レベルの品質を目指す完全リライト。
・Variance Adaptor（ピッチ/エネルギー残差予測）
・Dilated Residual PostNet（広受容野スペクトル補正）
・Multi-Resolution STFT Loss / LogMel Loss
・手動DSP最小化（モデル出力をそのままWORLDに渡す設計）
・WORLD ボコーダー使用（ボコーダー学習不要）
"""

# ==========================================
# グローバル設定
# ==========================================
SAMPLE_RATE     = 44100
FRAME_PERIOD_MS = 5.0
MCEP_DIMENSION  = 128

try:
    AP_DIMENSION = pw.get_num_aperiodicities(SAMPLE_RATE)
except AttributeError:
    AP_DIMENSION = 5

# ==========================================
# 日本語音素辞書
# ==========================================
def create_phoneme_map():
    phonemes_list = [
        'SP', 'AP', 'N', 'a', 'b', 'by', 'ch', 'cl', 'd', 'dy', 'e', 'f',
        'g', 'gy', 'h', 'hy', 'i', 'j', 'k', 'ky', 'm', 'my', 'n', 'ny',
        'o', 'p', 'py', 'r', 'ry', 's', 'sh', 't', 'ts', 'u', 'w', 'y',
        'z', 'br', 'pau'
    ]
    phonemes = set(phonemes_list)
    sorted_p = sorted(list(phonemes - {'SP', 'AP', 'pau'}))
    phoneme_map = {'PAD': 0, 'SP': 1, 'AP': 2}
    for i, p in enumerate(sorted_p):
        phoneme_map[p] = i + 3
    return phoneme_map, len(phoneme_map)

PHONEME_MAP, NUM_PHONEMES = create_phoneme_map()
CONSONANTS = {
    'b','by','ch','cl','d','dy','f','g','gy','h','hy','j',
    'k','ky','m','my','n','ny','p','py','r','ry','s','sh',
    't','ts','w','y','z','br','N'
}
VOWELS = {'a', 'i', 'u', 'e', 'o'}

def is_consonant(p): return p in CONSONANTS
def is_vowel(p):     return p in VOWELS

# ==========================================
# AIモデル定義（v9.0 プロ品質版）
# ==========================================

class Swish(nn.Module):
    def forward(self, x): return x * torch.sigmoid(x)


class RelativePositionEncoding(nn.Module):
    def __init__(self, dim, max_len=4096):
        super().__init__()
        pe  = torch.zeros(max_len, dim)
        pos = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, dim, 2).float() * (-np.log(10000.0) / dim))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


class ConformerBlock(nn.Module):
    """ConformerBlock with FiLM speaker conditioning."""
    def __init__(self, dim, heads=8, kernel_size=31, dropout=0.1, ff_mult=4):
        super().__init__()
        ff_dim = dim * ff_mult
        self.ff1 = nn.Sequential(
            nn.LayerNorm(dim), nn.Linear(dim, ff_dim), Swish(),
            nn.Dropout(dropout), nn.Linear(ff_dim, dim), nn.Dropout(dropout)
        )
        self.attn      = nn.MultiheadAttention(dim, heads, dropout=dropout, batch_first=True)
        self.norm_attn = nn.LayerNorm(dim)
        self.norm_conv = nn.LayerNorm(dim)
        self.conv = nn.Sequential(
            nn.Conv1d(dim, dim * 2, 1), nn.GLU(dim=1),
            nn.Conv1d(dim, dim, kernel_size, padding=kernel_size // 2, groups=dim),
            nn.BatchNorm1d(dim), Swish(),
            nn.Conv1d(dim, dim, 1), nn.Dropout(dropout)
        )
        self.film_gamma = nn.Linear(dim, dim)
        self.film_beta  = nn.Linear(dim, dim)
        self.ff2 = nn.Sequential(
            nn.LayerNorm(dim), nn.Linear(dim, ff_dim), Swish(),
            nn.Dropout(dropout), nn.Linear(ff_dim, dim), nn.Dropout(dropout)
        )
        self.norm_end = nn.LayerNorm(dim)

    def forward(self, x, spk_emb=None):
        x = x + 0.5 * self.ff1(x)
        if spk_emb is not None:
            g = self.film_gamma(spk_emb).unsqueeze(1)
            b = self.film_beta(spk_emb).unsqueeze(1)
            x = x * (1 + g) + b
        res = x
        xn  = self.norm_attn(x)
        x, _ = self.attn(xn, xn, xn)
        x   = res + x
        res = x
        xc  = self.norm_conv(x).transpose(1, 2)
        xc  = self.conv(xc).transpose(1, 2)
        x   = res + xc
        x   = x + 0.5 * self.ff2(x)
        return self.norm_end(x)


class ResBlock(nn.Module):
    def __init__(self, channels, kernel_size=3, dilation=1, dropout=0.05):
        super().__init__()
        pad        = dilation * (kernel_size - 1) // 2
        self.conv1 = weight_norm(nn.Conv1d(channels, channels, kernel_size,
                                           padding=pad, dilation=dilation))
        self.bn1   = nn.BatchNorm1d(channels)
        self.conv2 = weight_norm(nn.Conv1d(channels, channels, kernel_size,
                                           padding=kernel_size // 2))
        self.bn2   = nn.BatchNorm1d(channels)
        self.drop  = nn.Dropout(dropout)

    def forward(self, x):
        res = x
        x   = F.leaky_relu(self.bn1(self.conv1(x)), 0.1)
        x   = self.drop(x)
        x   = self.bn2(self.conv2(x))
        return F.leaky_relu(x + res, 0.1)


class DilatedPostNet(nn.Module):
    """
    Dilated Residual PostNet — 広い受容野でスペクトル包絡を精緻に補正。
    5層のDilated Conv (dilation 1,2,4,8,1) で受容野を大幅拡大。
    """
    def __init__(self, in_dim, hidden_dim=512, kernel_size=5, dropout=0.1):
        super().__init__()
        self.pre = nn.Sequential(
            weight_norm(nn.Conv1d(in_dim, hidden_dim, 1)),
            nn.LeakyReLU(0.1)
        )
        dilations = [1, 2, 4, 8, 1]
        self.blocks = nn.ModuleList()
        for d in dilations:
            pad = d * (kernel_size - 1) // 2
            self.blocks.append(nn.Sequential(
                nn.LeakyReLU(0.1),
                weight_norm(nn.Conv1d(hidden_dim, hidden_dim, kernel_size,
                                      dilation=d, padding=pad)),
                nn.LeakyReLU(0.1),
                nn.Dropout(dropout),
                weight_norm(nn.Conv1d(hidden_dim, hidden_dim, 1)),
            ))
        self.post = weight_norm(nn.Conv1d(hidden_dim, in_dim, 1))

    def forward(self, x):
        # x: (B, T, C)
        res_orig = x
        x = x.transpose(1, 2)  # -> (B, C, T)
        x = self.pre(x)
        for blk in self.blocks:
            x = x + blk(x)     # residual
        x = self.post(x)
        x = x.transpose(1, 2)  # -> (B, T, C)
        return res_orig + x


class VariancePredictor(nn.Module):
    """軽量 Variance Predictor (2-layer Conv1D)"""
    def __init__(self, hidden_dim, out_dim=1, kernel_size=3, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size, padding=kernel_size//2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size, padding=kernel_size//2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
        )
        self.proj = nn.Linear(hidden_dim, out_dim)

    def forward(self, x):
        # x: (B, T, C)
        out = self.net(x.transpose(1, 2)).transpose(1, 2)
        return self.proj(out)


class SingingModel(nn.Module):
    """
    v9.0 Generator: ResNet → Conformer×2 → BiLSTM → Conformer×2 →
                     VarianceAdaptor(energy) → Heads + DilatedPostNet
    """
    def __init__(
        self,
        num_phonemes: int  = NUM_PHONEMES,
        mcep_dim: int      = MCEP_DIMENSION,
        ap_dim: int        = AP_DIMENSION,
        embedding_dim: int = 384,
        hidden_dim: int    = 512,
        num_speakers: int  = 128,
    ):
        super().__init__()
        self.phoneme_embedding = nn.Embedding(num_phonemes, embedding_dim, padding_idx=0)
        self.speaker_embedding = nn.Embedding(num_speakers, hidden_dim)
        self.pitch_embedding   = nn.Embedding(128, hidden_dim)
        self.pos_enc           = RelativePositionEncoding(hidden_dim)

        # 入力: prev/curr/next phoneme emb + 3 numeric features + 1 energy
        input_dim = embedding_dim * 3 + 3
        self.pre_conv = nn.Sequential(
            weight_norm(nn.Conv1d(input_dim, hidden_dim, 3, padding=1)),
            nn.BatchNorm1d(hidden_dim), nn.LeakyReLU(0.1)
        )
        self.res_blocks = nn.ModuleList([
            ResBlock(hidden_dim, 3, 1), ResBlock(hidden_dim, 3, 2),
            ResBlock(hidden_dim, 3, 4), ResBlock(hidden_dim, 3, 8),
        ])
        self.conf_pre = nn.ModuleList([
            ConformerBlock(hidden_dim, heads=8, kernel_size=15, ff_mult=4),
            ConformerBlock(hidden_dim, heads=8, kernel_size=31, ff_mult=4),
        ])
        self.lstm = nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim // 2,
                            num_layers=3, batch_first=True,
                            bidirectional=True, dropout=0.1)
        self.conf_post = nn.ModuleList([
            ConformerBlock(hidden_dim, heads=8, kernel_size=15, ff_mult=4),
            ConformerBlock(hidden_dim, heads=8, kernel_size=31, ff_mult=4),
        ])

        # Variance Predictor (energy residual)
        self.energy_predictor = VariancePredictor(hidden_dim, out_dim=1)

        # Output heads (3-layer MLP for better expressiveness)
        self.mgc_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), Swish(), nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim // 2), Swish(),
            nn.Linear(hidden_dim // 2, mcep_dim)
        )
        self.lf0_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2), Swish(),
            nn.Linear(hidden_dim // 2, hidden_dim // 4), Swish(),
            nn.Linear(hidden_dim // 4, 1)
        )
        self.vuv_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 4), Swish(),
            nn.Linear(hidden_dim // 4, 1)
        )
        self.bap_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 4), Swish(),
            nn.Linear(hidden_dim // 4, ap_dim)
        )

        # Dilated Residual PostNets
        self.mgc_postnet = DilatedPostNet(mcep_dim, hidden_dim=256, kernel_size=5, dropout=0.1)
        self.bap_postnet = DilatedPostNet(ap_dim,   hidden_dim=128, kernel_size=3, dropout=0.1)

    def forward(self, ph_ids, feats, spk_id=None, pitch_id=None):
        prev_e = self.phoneme_embedding(ph_ids[:, :, 0])
        curr_e = self.phoneme_embedding(ph_ids[:, :, 1])
        next_e = self.phoneme_embedding(ph_ids[:, :, 2])
        x      = torch.cat([prev_e, curr_e, next_e, feats], dim=-1)

        bs = x.shape[0]
        if spk_id is None:
            spk_id = torch.zeros(bs, device=x.device, dtype=torch.long)
        spk_emb = self.speaker_embedding(spk_id)

        x = x.transpose(1, 2)
        x = self.pre_conv(x)
        for blk in self.res_blocks:
            x = blk(x)
        x = x.transpose(1, 2)

        if pitch_id is not None:
            x = x + self.pitch_embedding(pitch_id.clamp(0, 127))

        x = self.pos_enc(x)
        for blk in self.conf_pre:
            x = blk(x, spk_emb)
        x, _ = self.lstm(x)
        for blk in self.conf_post:
            x = blk(x, spk_emb)

        # Variance prediction
        energy_pred = self.energy_predictor(x)

        # Output heads
        mgc_pre  = self.mgc_head(x)
        lf0      = self.lf0_head(x)
        vuv      = self.vuv_head(x)
        bap_pre  = self.bap_head(x)

        # PostNet refinement
        mgc_post = self.mgc_postnet(mgc_pre)
        bap_post = self.bap_postnet(bap_pre)

        return mgc_pre, mgc_post, lf0, vuv, bap_pre, bap_post, energy_pred


# ==========================================
# モデルロードユーティリティ
# ==========================================

def load_model_weights_safe(model, state_dict, log_func=None):
    def _log(m):
        if log_func: log_func(m)
        else: print(m)

    new_dict = {}
    for k, v in state_dict.items():
        if k.startswith("conformer.0."):
            new_dict[k.replace("conformer.0.", "conf_pre.0.", 1)] = v
        elif k.startswith("conformer.1."):
            new_dict[k.replace("conformer.1.", "conf_pre.1.", 1)] = v
        elif k.startswith("conf1."):
            new_dict[k.replace("conf1.", "conf_pre.0.", 1)] = v
        elif k.startswith("conf2."):
            new_dict[k.replace("conf2.", "conf_pre.1.", 1)] = v
        else:
            new_dict[k] = v

    final_dict = {}
    for k, v in new_dict.items():
        if "parametrizations." in k and ".original" in k:
            parts = k.split('.')
            try:
                idx     = parts.index('parametrizations')
                base    = ".".join(parts[:idx])
                pname   = parts[idx + 1]
                suffix  = parts[-1]
                new_key = f"{base}.{pname}_g" if suffix.endswith('0') else f"{base}.{pname}_v"
                final_dict[new_key] = v
            except (ValueError, IndexError):
                final_dict[k] = v
        else:
            final_dict[k] = v

    model_state = model.state_dict()
    missing = [k for k in model_state if k not in final_dict]
    if missing:
        _log(f"⚠️ Missing {len(missing)} keys (新規初期化)")

    try:
        model.load_state_dict(final_dict, strict=False)
        _log("✅ モデルロード成功")
    except RuntimeError as e:
        _log(f"❌ ロード失敗: {e}")
        raise


# ==========================================
# GV ユーティリティ
# ==========================================

def calculate_gv(data):
    return np.mean(data, axis=0), np.var(data, axis=0)

def apply_gv(gen, gv_mean, gv_var, alpha=1.0):
    gm    = np.mean(gen, axis=0)
    gs    = np.std(gen, axis=0)
    ts    = np.sqrt(np.maximum(gv_var, 1e-8))
    gs    = np.maximum(gs, 1e-8)
    ratio = ts / gs
    return gm + (gen - gm) * (ratio ** alpha)


# ==========================================
# 音声処理関数群（v9.0 最小DSP版）
# ==========================================

def get_fft_size(sr):
    if sr >= 44100: return 4096
    elif sr >= 24000: return 2048
    return 1024


def apply_high_pass_filter(y, sr, cutoff=60.0):
    """HPF — 低域ランブル除去"""
    sos = signal.butter(4, cutoff, 'hp', fs=sr, output='sos')
    return signal.sosfilt(sos, y)


def multiband_compress(y, sr):
    """マルチバンドコンプレッション（軽量版）"""
    sos_lo  = signal.butter(4, 200,          'lp', fs=sr, output='sos')
    sos_mid = signal.butter(4, [200, 2000],  'bp', fs=sr, output='sos')
    sos_hi  = signal.butter(4, [2000, 8000], 'bp', fs=sr, output='sos')
    sos_air = signal.butter(4, 8000,         'hp', fs=sr, output='sos')
    lo  = signal.sosfilt(sos_lo,  y)
    mid = signal.sosfilt(sos_mid, y)
    hi  = signal.sosfilt(sos_hi,  y)
    air = signal.sosfilt(sos_air, y)

    def rms_compress(band, thr=0.3, ratio=2.5, makeup=1.0):
        rms  = np.sqrt(np.mean(band ** 2) + 1e-12)
        gain = thr / rms * (rms / thr) ** (1.0 / ratio) if rms > thr else 1.0
        return band * gain * makeup

    lo  = rms_compress(lo,  thr=0.55, ratio=3.0, makeup=0.85)
    mid = rms_compress(mid, thr=0.4,  ratio=3.0, makeup=1.0)
    hi  = rms_compress(hi,  thr=0.35, ratio=3.5, makeup=1.0)
    air = rms_compress(air, thr=0.2,  ratio=2.0, makeup=0.95)
    return lo + mid + hi + air


def soft_limiter(y, threshold=0.92):
    """ソフトリミッター"""
    y    = y - np.mean(y)
    peak = np.max(np.abs(y))
    if peak < threshold:
        return y
    y = np.tanh(y / threshold * 1.05) * threshold * 0.97
    return y


def normalize_f0(f0_hz):
    lf0 = np.zeros_like(f0_hz, dtype=np.float32)
    v   = f0_hz > 0
    if v.any(): lf0[v] = np.log(f0_hz[v])
    return lf0


def synthesis_wav(f0, mcep, ap, sr, high_quality=True,
                  vuv=None, formant_strength=1.0,
                  breath_intensity=0.0015,
                  **kwargs) -> np.ndarray:
    """
    v9.0 高品質歌声合成パイプライン
    設計思想: モデルが品質を担保するため、手動DSPを最小限に。
    WORLDに直接渡して合成する。
    """
    fft_size = get_fft_size(sr)
    f0 = f0.astype(np.float64)
    f0 = np.nan_to_num(f0, nan=0.0, posinf=2000.0, neginf=0.0)
    f0 = np.clip(f0, 0.0, 2000.0)

    # スペクトル包絡デコード
    sp = pw.decode_spectral_envelope(mcep.astype(np.float64), sr, fft_size)
    sp = np.maximum(sp, 1e-14)
    sp = np.nan_to_num(sp, nan=1e-14, posinf=1e-14, neginf=1e-14)

    # 異常値クリッピング
    sp_max = np.percentile(sp, 99.9)
    sp = np.clip(sp, 1e-14, max(sp_max, 1e-6))

    # 非周期成分デコード
    ap_full = pw.decode_aperiodicity(ap.astype(np.float64), sr, fft_size)
    ap_full = np.clip(ap_full, 0.0, 0.999)
    ap_full = np.nan_to_num(ap_full, nan=0.0)

    # WORLD合成
    f0      = np.ascontiguousarray(f0)
    sp      = np.ascontiguousarray(sp)
    ap_full = np.ascontiguousarray(ap_full)

    y = pw.synthesize(f0, sp, ap_full, sr, frame_period=FRAME_PERIOD_MS)

    if high_quality:
        # 息継ぎ音（自然さのため）
        if breath_intensity > 0 and vuv is not None:
            unvoiced = np.zeros(len(f0), dtype=bool)
            uv = vuv.flatten() if hasattr(vuv, 'flatten') else np.array(vuv)
            unvoiced[:len(uv)] = uv[:len(unvoiced)] < 0.5
            if np.any(unvoiced):
                breath    = np.random.randn(len(y)) * breath_intensity
                sos       = signal.butter(6, [100, 8000], 'bp', fs=sr, output='sos')
                breath    = signal.sosfilt(sos, breath)
                uv_interp = np.interp(
                    np.arange(len(y)),
                    np.linspace(0, len(y) - 1, len(f0)),
                    unvoiced.astype(float)
                )
                uv_interp = gaussian_filter1d(uv_interp, 5.0)
                y = y + breath * uv_interp

        # HPF
        y = apply_high_pass_filter(y, sr, cutoff=60.0)

    # 正規化
    y    = np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)
    y    = y - np.mean(y)
    peak = np.max(np.abs(y))
    if peak > 1e-6:
        y = y / peak
    y = np.clip(y, -1.0, 1.0)

    return y.astype(np.float32)


# ==========================================
# G2P / UST パース
# ==========================================

def g2p_kana(kana):
    """かな → 音素列 (内包した g2p_manual を使用)"""
    if not kana or kana.lower() == 'r': return ['SP']
    try:
        phs = g2p_manual(kana)  # ← 上で定義した kana_map の g2p_manual を直接呼ぶ
        out = []
        for p in phs:
            if p == 'pau':         out.append('SP')
            elif p == 'cl':        out.append('cl')
            elif p in PHONEME_MAP: out.append(p)
        return out if out else ['SP']
    except:
        return ['SP']


def parse_ust(ust_file):
    notes, tempo = [], 120.0
    try:
        with open(ust_file, 'r', encoding='cp932') as f:
            lines = f.readlines()
    except:
        try:
            with open(ust_file, 'r', encoding='utf-8') as f:
                lines = f.readlines()
        except:
            return notes, tempo

    curr, in_sec, current_tempo = {}, False, 120.0
    for line in lines:
        line = line.strip()
        if not line: continue
        if line.startswith('[#'):
            if in_sec and 'Lyric' in curr:
                curr.setdefault('Tempo', current_tempo)
                notes.append(curr)
            in_sec = True; curr = {}
        elif line.startswith('Tempo='):
            try: current_tempo = float(line.split('=')[1])
            except: pass
        elif in_sec and '=' in line:
            k, v = line.split('=', 1)
            curr[k] = v

    if in_sec and 'Lyric' in curr:
        curr.setdefault('Tempo', current_tempo)
        notes.append(curr)

    return notes, tempo


def decode_ust_pitch(note, dur_frames, frame_period_ms):
    """USTのPitchBend + VBRデコード"""
    curve  = np.zeros(dur_frames, dtype=np.float32)
    pb_str = note.get('PitchBend', '')
    if pb_str:
        try:
            pb = [float(x) for x in pb_str.split(',') if x]
            if pb:
                idxs   = np.linspace(0, len(pb) - 1, dur_frames)
                curve += np.interp(idxs, np.arange(len(pb)), pb).astype(np.float32)
        except: pass

    vbr_str = note.get('VBR', '')
    if vbr_str:
        try:
            vp = [float(x) for x in vbr_str.split(',')]
            if len(vp) >= 3 and vp[0] > 0:
                vlen   = vp[0] / 100.0
                period = vp[1]
                depth  = vp[2]
                vin    = vp[3] / 100.0 if len(vp) > 3 else 0.1
                vout   = vp[4] / 100.0 if len(vp) > 4 else 0.1
                phase  = vp[5] if len(vp) > 5 else 0.0
                offset = vp[6] if len(vp) > 6 else 0.0
                vf     = int(dur_frames * vlen)
                sf_    = dur_frames - vf
                if sf_ < 0: sf_ = 0
                cf     = dur_frames - sf_
                if cf > 0:
                    t   = np.arange(cf) * frame_period_ms
                    vib = (np.sin(2 * np.pi * t / period +
                                  phase / 100.0 * 2 * np.pi) * depth + offset)
                    env = np.ones(cf)
                    fin = int(cf * vin); fout = int(cf * vout)
                    if fin  > 0: env[:fin]   = np.linspace(0, 1, fin)
                    if fout > 0: env[-fout:] = np.linspace(1, 0, fout)
                    curve[sf_:] += vib * env
        except: pass

    return curve


def ust_to_features(notes, default_tempo):
    """UST → 特徴量変換"""
    timed_ph, last_vowel = [], None
    for note in notes:
        try:
            length = float(note.get('Length', 480))
            lyric  = note.get('Lyric', 'r')
            num    = int(note.get('NoteNum', 60))
            tempo  = float(note.get('Tempo', default_tempo))
        except: continue

        ms_per_tick = 60000 / (tempo * 480)
        dur_ms = length * ms_per_tick
        f0_hz  = 440.0 * (2.0 ** ((num - 69) / 12.0))
        lf0    = np.log(f0_hz) if lyric != 'r' else 0.0
        pid    = num if lyric != 'r' else 0

        if lyric.strip() in ('+', '-', '_'):
            phs = [last_vowel] if last_vowel and lf0 != 0.0 else ['SP']
        elif lyric.lower() == 'r':
            phs = ['SP']
        else:
            phs = g2p_kana(lyric)

        if not phs: continue
        ph_dur = dur_ms / len(phs)
        for p in phs:
            timed_ph.append({'p': p, 'dur_ms': ph_dur, 'lf0': lf0,
                              'pitch_id': pid, 'note_raw': note})
        cv = next((p for p in reversed(phs) if p in VOWELS), None)
        if cv and lyric.strip() not in ('+', '-', '_'):
            last_vowel = cv

    if not timed_ph:
        return np.array([]), np.array([]), np.array([])

    total_ms   = sum(t['dur_ms'] for t in timed_ph)
    tf         = int(total_ms / FRAME_PERIOD_MS) + 100
    ids        = np.zeros((tf, 3), dtype=np.int64)
    nums       = np.zeros((tf, 3), dtype=np.float32)
    p_ids      = np.zeros(tf, dtype=np.int64)
    ph_id_list = [PHONEME_MAP.get(t['p'], PHONEME_MAP['SP']) for t in timed_ph]

    cur_ms, last_f, cur_f = 0.0, 0, 0
    for i, tp in enumerate(timed_ph):
        end_ms = cur_ms + tp['dur_ms']
        s = last_f
        e = min(max(s + 1, int(end_ms / FRAME_PERIOD_MS)), tf)
        if s >= tf: break

        ids[s:e, 0] = ph_id_list[i-1] if i > 0 else PHONEME_MAP['SP']
        ids[s:e, 1] = ph_id_list[i]
        ids[s:e, 2] = ph_id_list[i+1] if i < len(ph_id_list)-1 else PHONEME_MAP['SP']

        frames = e - s
        if frames > 0:
            nums[s:e, 0] = np.arange(frames) / frames
            if 'note_raw' in tp and tp['note_raw']:
                dev = decode_ust_pitch(tp['note_raw'], frames, FRAME_PERIOD_MS)
                nums[s:e, 2] = tp['lf0'] + (dev / 1200.0) * np.log(2.0)
            else:
                nums[s:e, 2] = tp['lf0']
        nums[s:e, 1] = frames
        p_ids[s:e]   = tp['pitch_id']
        cur_ms = end_ms; last_f = e; cur_f = e

    return ids[:cur_f], nums[:cur_f], p_ids[:cur_f]


def ust_to_features_fixed_consonant(notes, default_tempo, consonant_length_ms=60.0):
    """子音固定長モード"""
    segs, last_vowel = [], None
    for note in notes:
        try:
            length = float(note.get('Length', 480))
            lyric  = note.get('Lyric', 'r')
            num    = int(note.get('NoteNum', 60))
            tempo  = float(note.get('Tempo', default_tempo))
        except: continue

        ms_per_tick = 60000 / (tempo * 480)
        dur_ms = length * ms_per_tick
        f0_hz  = 440.0 * (2.0 ** ((num - 69) / 12.0))
        lf0    = np.log(f0_hz) if lyric != 'r' else 0.0
        pid    = num if lyric != 'r' else 0
        phs, durs, cv = [], [], None

        if lyric.strip() in ('+', '-', '_'):
            if last_vowel and lf0 != 0.0:
                phs, durs, cv = [last_vowel], [dur_ms], last_vowel
            else:
                phs, durs = ['SP'], [dur_ms]
        elif lyric.lower() == 'r':
            phs, durs = ['SP'], [dur_ms]
        else:
            phs_raw = g2p_kana(lyric)
            if not phs_raw:
                phs, durs = ['SP'], [dur_ms]
            else:
                phs   = phs_raw
                cons  = [p for p in phs if is_consonant(p)]
                vows  = [p for p in phs if not is_consonant(p)]
                c_len = min(consonant_length_ms,
                            (dur_ms * 0.65) / max(len(cons), 1))
                rem   = dur_ms - len(cons) * c_len
                v_len = rem / max(len(vows), 1)
                assigned = 0.0
                for p in phs:
                    d = c_len if is_consonant(p) else v_len
                    durs.append(d); assigned += d
                if durs: durs[-1] += (dur_ms - assigned)
                cv = next((p for p in reversed(phs) if p in VOWELS), None)

        if phs:
            segs.append({'phs': phs, 'durs': durs, 'lf0': lf0,
                         'pitch_id': pid, 'note_raw': note})
            if cv and lyric.strip() not in ('+', '-', '_'):
                last_vowel = cv

    if not segs:
        return np.array([]), np.array([]), np.array([])

    total_ms = sum(sum(s['durs']) for s in segs)
    tf       = int(total_ms / FRAME_PERIOD_MS) + 100
    ids      = np.zeros((tf, 3), dtype=np.int64)
    nums     = np.zeros((tf, 3), dtype=np.float32)
    p_ids    = np.zeros(tf, dtype=np.int64)

    all_p, all_d, all_l, all_pi, all_notes = [], [], [], [], []
    for s in segs:
        all_p.extend(s['phs']);  all_d.extend(s['durs'])
        all_l.extend([s['lf0']] * len(s['phs']))
        all_pi.extend([s['pitch_id']] * len(s['phs']))
        for j in range(len(s['phs'])):
            all_notes.append(s['note_raw'] if j == 0 else None)

    ph_id_list            = [PHONEME_MAP.get(p, PHONEME_MAP['SP']) for p in all_p]
    cur_ms, last_f, cur_f = 0.0, 0, 0

    for i, (p, d_ms, lf0, pi, note_raw) in enumerate(
            zip(all_p, all_d, all_l, all_pi, all_notes)):
        end_ms = cur_ms + d_ms
        s = last_f
        e = min(max(s + 1, int(end_ms / FRAME_PERIOD_MS)), tf)
        if s >= tf: break

        ids[s:e, 0] = ph_id_list[i-1] if i > 0 else PHONEME_MAP['SP']
        ids[s:e, 1] = ph_id_list[i]
        ids[s:e, 2] = ph_id_list[i+1] if i < len(ph_id_list)-1 else PHONEME_MAP['SP']

        frames = e - s
        if frames > 0:
            nums[s:e, 0] = np.arange(frames) / frames
            if note_raw is not None:
                dev = decode_ust_pitch(note_raw, frames, FRAME_PERIOD_MS)
                nums[s:e, 2] = lf0 + (dev / 1200.0) * np.log(2.0)
            else:
                nums[s:e, 2] = lf0
        nums[s:e, 1] = frames
        p_ids[s:e]   = pi
        cur_ms = end_ms; last_f = e; cur_f = e

    return ids[:cur_f], nums[:cur_f], p_ids[:cur_f]


# =============================================================================
# 5. モデルキャッシュ / チャンク推論
# =============================================================================
_cached_path   = None
_cached_model  = None
_cached_scaler = None
_device        = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 デバイス: {_device}")


def _load_model(model_path: str):
    global _cached_path, _cached_model, _cached_scaler

    model_path = model_path.strip()
    if _cached_path == model_path and _cached_model is not None:
        return _cached_model, _cached_scaler, "（キャッシュから読み込み）"

    logs = []

    if model_path.lower().endswith(".yvs"):
        logs.append("ℹ️ 統合モデルファイル (.yvs) を読み込んでいます...")
        try:
            ckpt = torch.load(model_path, map_location="cpu", weights_only=False)
        except TypeError:
            ckpt = torch.load(model_path, map_location="cpu")

        if not (isinstance(ckpt, dict) and "model_state_dict" in ckpt):
            raise ValueError("未対応または破損した .yvs ファイルです。")

        sc         = ckpt["scaler"]
        state_dict = ckpt["model_state_dict"]
        config     = ckpt.get("config", {})
        hidden_dim = config.get("hidden_dim", 512)
        mtype      = config.get("model_type", "unknown")

        if mtype == "base":
            logs.append("✅ V1 汎用モデルを検出")
        elif mtype == "adapted":
            logs.append(f"✅ V2 話者適応モデルを検出 (ベース: {config.get('base_model','?')})")
        else:
            logs.append("✅ モデル読み込み完了")
    else:
        logs.append("⚠️ 従来の .pth ファイルを読み込みます。")
        s_path = model_path.replace(".pth", "_scaler.pkl")
        if not os.path.exists(s_path):
            s_path = model_path.replace(".pth", ".pyl")
        with open(s_path, "rb") as f:
            sc = pickle.load(f)
        try:
            state_dict = torch.load(model_path, map_location="cpu", weights_only=False)
        except TypeError:
            state_dict = torch.load(model_path, map_location="cpu")
        hidden_dim = 512

    logs.append("🤖 v9.0 モデルをロード中...")
    model = SingingModel(hidden_dim=hidden_dim)
    load_model_weights_safe(model, state_dict, log_func=lambda m: logs.append(m))
    model.eval()

    try:
        model = model.to(_device)
    except RuntimeError as re:
        if "out of memory" in str(re):
            logs.append("⚠️ GPU メモリ不足のため CPU モードに切り替えます")
            model = model.to(torch.device("cpu"))
            torch.cuda.empty_cache()
        else:
            raise

    _cached_path   = model_path
    _cached_model  = model
    _cached_scaler = sc
    return model, sc, "\n".join(logs)


def _inference_chunked(model, ids, nums, p_ids, chunk_size=1500, overlap=300):
    """チャンク推論 (v9.0: 7出力対応・ハニング窓ブレンド)"""
    total = ids.shape[0]

    def _forward(id_t, nu_t, pi_t):
        mgc_pre, mgc_post, lf0, vuv, bap_pre, bap_post, energy = model(id_t, nu_t, pitch_id=pi_t)
        return (
            mgc_post.squeeze(0).cpu().numpy(),
            lf0.squeeze(0).cpu().numpy(),
            torch.sigmoid(vuv.squeeze(0)).cpu().numpy(),
            bap_post.squeeze(0).cpu().numpy(),
        )

    if total <= chunk_size:
        with torch.no_grad():
            return _forward(
                torch.LongTensor(ids).unsqueeze(0).to(_device),
                torch.FloatTensor(nums).unsqueeze(0).to(_device),
                torch.LongTensor(p_ids).unsqueeze(0).to(_device),
            )

    step    = chunk_size - overlap
    mgc_acc = np.zeros((total, MCEP_DIMENSION), dtype=np.float32)
    lf0_acc = np.zeros((total, 1),              dtype=np.float32)
    vuv_acc = np.zeros((total, 1),              dtype=np.float32)
    bap_acc = np.zeros((total, AP_DIMENSION),   dtype=np.float32)
    weight  = np.zeros(total,                    dtype=np.float32)

    hann     = np.hanning(overlap * 2).astype(np.float32)
    fade_in  = hann[:overlap]
    fade_out = hann[overlap:]

    model.eval()
    with torch.no_grad():
        start = 0
        while start < total:
            end = min(start + chunk_size, total)
            m, l, v, b = _forward(
                torch.LongTensor(ids[start:end]).unsqueeze(0).to(_device),
                torch.FloatTensor(nums[start:end]).unsqueeze(0).to(_device),
                torch.LongTensor(p_ids[start:end]).unsqueeze(0).to(_device),
            )
            L = end - start
            w = np.ones(L, dtype=np.float32)
            if start > 0:
                fi = min(overlap, L); w[:fi]  *= fade_in[:fi]
            if end < total:
                fo = min(overlap, L); w[-fo:] *= fade_out[-fo:]

            mgc_acc[start:end] += m * w[:, np.newaxis]
            lf0_acc[start:end] += l * w[:, np.newaxis]
            vuv_acc[start:end] += v * w[:, np.newaxis]
            bap_acc[start:end] += b * w[:, np.newaxis]
            weight[start:end]  += w
            start += step

    ws = np.maximum(weight, 1e-8)
    return (mgc_acc / ws[:, None], lf0_acc / ws[:, None],
            vuv_acc / ws[:, None], bap_acc / ws[:, None])


# =============================================================================
# 6. メイン推論関数 (Gradio コールバック)
# =============================================================================
def run_inference(
    singer_name, ust_file,
    pitch_correction, gv_alpha, timbre_smooth, pitch_smooth,
    formant_strength, breath_intensity,
    use_multiband, volume_gain,
    consonant_mode, consonant_length,
    progress=gr.Progress(track_tqdm=True),
):
    logs = []
    def log(msg): logs.append(msg); print(msg)

    try:
        if not singer_name or singer_name not in MODEL_REGISTRY:
            raise ValueError(f"歌手が選択されていません。MODEL_REGISTRY にモデルを登録してください。")
        if ust_file is None:
            raise ValueError("UST ファイルがアップロードされていません。")

        yvs_path = MODEL_REGISTRY[singer_name]
        if not os.path.exists(yvs_path):
            raise FileNotFoundError(
                f"モデルファイルが見つかりません: {yvs_path}\n"
                f"Google Drive がマウントされているか確認してください。"
            )

        ust_path = ust_file if isinstance(ust_file, str) else ust_file.name

        log(f"👤 歌手: {singer_name}")
        log("📥 モデル読み込み中...")
        progress(0.05, desc="モデル読み込み中...")
        model, sc, load_log = _load_model(yvs_path)
        log(load_log)

        log("🎼 UST 解析中...")
        progress(0.15, desc="UST 解析中...")
        notes, tempo = parse_ust(ust_path)
        if not notes:
            raise ValueError("UST ファイルから有効なノートが取得できませんでした。")

        log("📝 ノート処理中...")
        progress(0.25, desc="ノート処理中...")
        if consonant_mode == "ON":
            log(f"  子音固定モード: {consonant_length:.0f}ms")
            ids, nums, p_ids = ust_to_features_fixed_consonant(
                notes, tempo, float(consonant_length))
        else:
            ids, nums, p_ids = ust_to_features(notes, tempo)

        if ids.size == 0:
            raise ValueError("UST から有効なノート情報が抽出できませんでした。")

        lf0_target  = nums[:, 2].copy()
        nums_scaled = sc["nums"].transform(nums)

        log("🧠 推論実行中 (v9.0 高品質モード)...")
        progress(0.40, desc="AI 推論中...")
        m_mgc, m_lf0, m_vuv, m_bap = _inference_chunked(
            model, ids, nums_scaled, p_ids, chunk_size=1500, overlap=300)

        log("🔄 後処理中...")
        progress(0.65, desc="後処理中...")

        mgc = sc["mgc"].inverse_transform(m_mgc)
        bap = sc["bap"].inverse_transform(m_bap)
        lf0 = m_lf0.flatten()
        vuv = m_vuv.flatten()

        if pitch_correction > 0:
            voiced = vuv > 0.5
            if np.any(voiced):
                n  = min(len(lf0_target), len(lf0))
                vm = voiced[:n]
                lf0[:n][vm] += (lf0_target[:n][vm] - lf0[:n][vm]) * pitch_correction

        if "gv_mgc" in sc and gv_alpha > 0:
            mgc = apply_gv(mgc, sc["gv_mgc"][0], sc["gv_mgc"][1], gv_alpha)

        if timbre_smooth > 0:
            for i in range(mgc.shape[1]):
                mgc[:, i] = gaussian_filter1d(mgc[:, i], timbre_smooth)

        lf0 = medfilt(lf0, kernel_size=3)
        if pitch_smooth > 0:
            lf0 = gaussian_filter1d(lf0, pitch_smooth)

        for i in range(bap.shape[1]):
            bap[:, i] = gaussian_filter1d(bap[:, i], 0.8)

        f0 = np.exp(np.where(lf0 > 0, lf0, 0))
        f0 = np.nan_to_num(f0, nan=0.0, posinf=1600.0, neginf=0.0)
        f0 = np.clip(f0, 0, 1600.0)

        vuv_smooth = gaussian_filter1d(vuv, 1.5)
        f0[vuv_smooth < 0.5] = 0.0

        log("🎵 レンダリング中 (WORLD 合成)...")
        progress(0.80, desc="WORLD 合成中...")
        y = synthesis_wav(
            f0, mgc, bap, SAMPLE_RATE,
            high_quality=True,
            vuv=vuv_smooth,
            formant_strength=float(formant_strength),
            breath_intensity=float(breath_intensity),
        )

        if use_multiband:
            log("  🎚️ マルチバンドコンプレッション適用中...")
            y    = multiband_compress(y, SAMPLE_RATE)
            peak = np.max(np.abs(y))
            if peak > 1e-6:
                y = y / peak

        y = y * float(volume_gain)
        y = np.nan_to_num(y, nan=0.0, posinf=1.0, neginf=-1.0)
        y = soft_limiter(y, threshold=0.92)

        progress(0.95, desc="WAV 書き出し中...")
        out_path = os.path.join(tempfile.gettempdir(), "singing_output.wav")
        sf.write(out_path, y.astype(np.float32), SAMPLE_RATE)

        log(f"✅ 完了: {len(y) / SAMPLE_RATE:.2f} 秒の音声を生成しました")
        progress(1.0, desc="完了")
        return out_path, "\n".join(logs)

    except Exception as e:
        log(f"❌ エラー: {e}")
        log(traceback.format_exc())
        return None, "\n".join(logs)


# =============================================================================
# 7. Gradio UI
# =============================================================================
with gr.Blocks(title="yui2024式 歌唱合成ソフト v9.0") as demo:

    gr.Markdown(
        """
        # 🎤 yui2024式 歌唱合成ソフト v9.0
        **Google Colab 対応 Gradio GUI — 完全単体版**
        > 歌手を選択し、UST をアップロードして「歌声生成開始」を押してください。
        """
    )

    with gr.Row():
        # ── 左カラム: 設定 ──────────────────────────────────────────────────
        with gr.Column(scale=1):

            gr.Markdown("### 📁 ファイル設定")
            singer_name = gr.Dropdown(
                choices=_MODEL_CHOICES,
                value=_MODEL_CHOICES[0] if _MODEL_CHOICES else None,
                label="🎤 歌手を選択",
                info="MODEL_REGISTRY にモデルを登録すると選択肢が増えます",
            )
            ust_file = gr.File(
                label="UST ファイル (楽譜) をアップロード",
                file_types=[".ust"],
            )

            gr.Markdown("### ⚙️ 音質調整")
            pitch_correction = gr.Slider(0.0, 1.0,   value=1.0,    step=0.01,   label="🎯 ピッチ補正強度")
            gv_alpha         = gr.Slider(0.0, 2.0,   value=1.0,    step=0.01,   label="GV強度 (メリハリ)")
            timbre_smooth    = gr.Slider(0.0, 3.0,   value=0.0,    step=0.1,    label="🎵 音色滑らかさ (σ)")
            pitch_smooth     = gr.Slider(0.0, 3.0,   value=0.0,    step=0.1,    label="🎶 音程滑らかさ (σ)")

            gr.Markdown("### ✨ 高度な音質設定")
            formant_strength = gr.Slider(0.0, 2.0,   value=1.0,    step=0.01,   label="🔊 フォルマント強調")
            breath_intensity = gr.Slider(0.0, 0.004, value=0.0015, step=0.0001, label="💨 息継ぎ音の強さ")

            gr.Markdown("### 🎚️ ダイナミクス処理")
            use_multiband = gr.Checkbox(label="マルチバンドコンプレッション", value=False)
            volume_gain   = gr.Slider(0.5, 1.0, value=0.88, step=0.01, label="音量ゲイン")

            gr.Markdown("### 🎵 子音長制御")
            consonant_mode   = gr.Radio(choices=["ON", "OFF"], value="ON", label="子音長固定モード")
            consonant_length = gr.Slider(20, 150, value=65, step=1, label="子音長 (ms)")

            run_btn = gr.Button("🎵 歌声生成開始", variant="primary", size="lg")

        # ── 右カラム: 出力 ──────────────────────────────────────────────────
        with gr.Column(scale=1):
            gr.Markdown("### 🎧 生成結果")
            audio_out = gr.Audio(
                label="生成音声（再生・ダウンロード）",
                type="filepath",
                interactive=False,
            )
            log_out = gr.Textbox(
                label="ログ",
                lines=25,
                max_lines=40,
                interactive=False,
                show_copy_button=True,
            )

    run_btn.click(
        fn=run_inference,
        inputs=[
            singer_name, ust_file,
            pitch_correction, gv_alpha, timbre_smooth, pitch_smooth,
            formant_strength, breath_intensity,
            use_multiband, volume_gain,
            consonant_mode, consonant_length,
        ],
        outputs=[audio_out, log_out],
    )

# =============================================================================
# 8. 起動 (share=True で公開 URL を発行)
# =============================================================================
demo.launch(share=True, debug=False)

🔧 デバイス: cuda
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b2638cfecae2a30ac5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyworld

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyworld: filename=pyworld-0.3.5-cp312-cp312-linux_x86_64.whl size=943059 sha256=d7c6f5480701e62be1428d4de4fc34c1719fbcbd55dfe6acb4056573048481d9
  Stored in directory: /root/.cache/pip/wheels/be/ac/58/c6a1791ec6d17f3a99b6ccdec92b472f560cb5c564b83dd77e
Successfully built pyworld
